<a href="https://colab.research.google.com/github/tek4vn/PPML/blob/main/HE_PATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
 
import torchvision
import torchvision.transforms as transforms
 
import os

In [ ]:
def polyfitX(x):
    return 0.1524*(x**2) + 0.5*x + 0.409
 
def polyfit(x):
    return x**2
 

class Polyfit(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return polyfitX(input)

In [ ]:
class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)
 
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out
 
    def _make_layers(self, cfg):
        layers = []
        in_channels = 1
        for x in cfg:
            if x == 'M':
                # Dùng AvgPool thay MaxPool khi dùng hàm xấp xỉ thay ReLU
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
                
                # layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
#                            Polyfit()]
                            nn.ReLU(inplace=True)]
                # layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                # nn.BatchNorm2d(x),
                # nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [ ]:
class VGG_HE(nn.Module):
    def __init__(self, vgg_name):
        super(VGG_HE, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)
 
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out
 
    def _make_layers(self, cfg):
        layers = []
        in_channels = 1
        for x in cfg:
            if x == 'M':
                # Dùng AvgPool thay MaxPool khi dùng hàm xấp xỉ thay ReLU
                layers += [nn.AvgPool2d(kernel_size=2, stride=2)]
                
                # layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           Polyfit()]
#                             nn.ReLU(inplace=True)]
                # layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                # nn.BatchNorm2d(x),
                # nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [ ]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

In [ ]:
transform_test = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

In [ ]:
trainset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform_train)
#trainloader_mnist = torch.utils.data.DataLoader(
#    trainset, batch_size=128, shuffle=True, num_workers=2)

In [ ]:
testset = torchvision.datasets.MNIST(
    root='./test', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)
device

In [ ]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [ ]:
X1, X2, student = torch.utils.data.random_split(trainset, [25000, 25000, 10000])

# Train

In [ ]:
trainloaderX1 = torch.utils.data.DataLoader(
    X1, batch_size=100, shuffle=True, num_workers=2)
trainloaderX2 = torch.utils.data.DataLoader(
    X2, batch_size=100, shuffle=True, num_workers=2)
# trainloaderX3 = torch.utils.data.DataLoader(
#     X3, batch_size=100, shuffle=True, num_workers=2)
# trainloaderX4 = torch.utils.data.DataLoader(
#     X4, batch_size=100, shuffle=True, num_workers=2)
# trainloaderX5 = torch.utils.data.DataLoader(
#     X5, batch_size=100, shuffle=True, num_workers=2)

In [ ]:
print('==> Building model..')
net = VGG('VGG13')
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, 
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
def train(epochs, dataload, model):
    
    for epoch in range(0, epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        print("Epoch",epoch)
        for batch_idx , (inputs, targets) in enumerate(dataload):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print('\tLoss: '+ str(train_loss/(batch_idx+1)),
              '\tAccuracy: '+ str(100. * (correct/total)),
              "\tCorrect/total: {correct}/{total}".format(correct=correct, total=total))

In [ ]:
train(15, trainloaderX1, net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, 
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Test

In [ ]:
def test(model, testload):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testload):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print("\nTest set: Avg. loss: {}\tAccuracy: {} \tCorrect/Total: {}/{}".format(
                test_loss/(batch_idx+1), 100. * correct/total,
                correct, total))
        

In [ ]:
test(net, testloader)

# Student

In [ ]:
from scipy import stats as s

In [ ]:
rd = torch.randint(0, 10000, (500,))
st500 = torch.utils.data.Subset(student, rd)

In [ ]:
rd = torch.randint(0, 10000, (1000,))
st1k = torch.utils.data.Subset(student, rd)

In [ ]:
rd = torch.randint(0, 10000, (2000,))
st2k = torch.utils.data.Subset(student, rd)

In [ ]:
rd = torch.randint(0, 10000, (5000,))
st5k = torch.utils.data.Subset(student, rd)

In [ ]:
rd = torch.randint(0, 10000, (7500,))
st7k5 = torch.utils.data.Subset(student, rd)

In [ ]:
lstData = [st500, st1k, st2k, st5k, st7k5, student]

In [ ]:
stloader_500 = torch.utils.data.DataLoader(
    st500, shuffle=False, num_workers=2)
stloader_1k = torch.utils.data.DataLoader(
    st1k, shuffle=False, num_workers=2)
stloader_2k = torch.utils.data.DataLoader(
    st2k, shuffle=False, num_workers=2)
stloader_5k = torch.utils.data.DataLoader(
    st5k, shuffle=False, num_workers=2)
stloader_7k5 = torch.utils.data.DataLoader(
    st7k5, shuffle=False, num_workers=2)
stloader = torch.utils.data.DataLoader(
    student, shuffle=False, num_workers=2)

In [ ]:
def pred(model1, model2, data):
    model1.eval()
    model2.eval()
#     model3.eval()
#     model4.eval()
#     model5.eval()
    predict_Mode = []
    with torch.no_grad():
        for data, target in data:
            data, target = data.to(device), target.to(device)
            lb1, lb2 = model1(data).argmax().item(), model2(data).argmax().item()
#             lb3 = model3(data).argmax().item()
#             lb4 = model4(data).argmax().item()
#             lb5 = model5(data).argmax().item()
            Modelb = s.mode([lb1, lb2]).mode
            predict_Mode.append([int(Modelb), target.item()])
    return predict_Mode

In [ ]:
lstTest = []
dict_data = {}
nameLoader = ['st500', 'st1k', 'st2k', 'st5k', 'st7k5', 'student']
lstLoader = [stloader_500, stloader_1k, stloader_2k, stloader_5k, stloader_7k5, stloader]
for d in lstLoader:
    label_test = pred(model1, model2, d )
    ptram = 0
    data_st = []
    pred_Test = []
    for i in range(len(label_test)):
        pred_Test.append(label_test[i][0])
        data_st.append([lstData[lstLoader.index(d)][i][0], label_test[i][0]])
        if label_test[i][0] == label_test[i][1]:
            ptram += 1
    dict_data[nameLoader[lstLoader.index(d)]] = data_st
    lstTest.append(ptram/len(label_test))

In [ ]:
dict_data['pred_Teacher'] = lstTest


In [ ]:
def pred_st(data_, he=False):
    __doc__ = """Lấy data và tạo model"""
    trainloader_st = torch.utils.data.DataLoader(
            dict_data[data_], batch_size=100, shuffle=True, num_workers=2)
    
    if he:
        print('==> Building model HE...')
        net = VGG_HE('VGG11')
        net = net.to(device)
        if device == 'cuda':
            net = torch.nn.DataParallel(net)
            cudnn.benchmark = True
        return net, trainloader_st
    else:
        print('==> Building model..')
        net = VGG('VGG11')
        net = net.to(device)
        if device == 'cuda':
            net = torch.nn.DataParallel(net)
            cudnn.benchmark = True
        return net, trainloader_st

In [ ]:
net, trainloader_st = pred_st('student', False)
# net, trainloader_st = pred_st('st500', True)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, 
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
train(25, trainloader_st, net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, 
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
train(25, trainloader_st, net)

In [ ]:
accSt = test(net, testloader)

# Visualize

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dict2 = torch.load('../input/dict-data/model2_teacher.data')
dict3 = torch.load('../input/dictdata/model3_dictData.data')
dict4 = torch.load('../input/dictdata/model4_dictData.data')
dict5 = torch.load('../input/dictdata/model5_dictData.data')

In [ ]:
dict2_HE = torch.load('../input/dictdata/model2_HE.data')
dict3_HE = torch.load('../input/dictdata/model3_HE.data')
dict4_HE = torch.load('../input/dictdata/model4_HE.data')
dict5_HE = torch.load('../input/dictdata/model5_HE.data')

In [ ]:
xlabel = [500, 1000, 2000, 5000, 7500, 10000]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,figsize=(18, 13))
fig.suptitle('Prediction vote', fontsize = 16)

ax2.plot(xlabel, dict2['pred_Teacher'], marker='o')
ax2.plot(xlabel, dict3['pred_Teacher'], marker='P')
ax2.plot(xlabel, dict4['pred_Teacher'], marker='v')
ax2.plot(xlabel, dict5['pred_Teacher'], marker='s')
ax2.set_ylabel('Not use HE', fontsize=14)
ax2.set_ylim(97, 100)
ax2.set_xticks(xlabel)

ax1.plot(xlabel, dict2_HE['pred_Teacher'], marker='o', label = '2 Teacher')
ax1.plot(xlabel, dict3_HE['pred_Teacher'], marker='P', label = '3 Teacher')
ax1.plot(xlabel, dict4_HE['pred_Teacher'], marker='v', label = '4 Teacher')
ax1.plot(xlabel, dict5_HE['pred_Teacher'], marker='s', label = '5 Teacher')
ax1.set_ylabel('Use HE', fontsize=14)
ax1.set_ylim(97, 100)
ax1.set_xticks(xlabel)
ax1.legend(loc = "lower right")


plt.xlabel('Samples', fontsize=16)
fig.savefig('Case_vote.jpg')
fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,figsize=(18, 13))
fig.suptitle("Student's prediction", fontsize = 16)

ax1.plot(xlabel, dict2['pred_Student_HE'], marker='o', label = '2 Teacher')
ax1.plot(xlabel, dict3['pred_Student_HE'], marker='P', label = '3 Teacher')
ax1.plot(xlabel, dict4['pred_Student_HE'], marker='v', label = '4 Teacher')
ax1.plot(xlabel, dict5['pred_Student_HE'], marker='s', label = '5 Teacher')
ax1.set_ylabel('Use HE', fontsize=14)
ax1.set_ylim(70, 100)
ax1.set_xticks(xlabel)
ax1.legend(loc = "lower right")

ax2.plot(xlabel, dict2['pred_Student'], marker='o')
ax2.plot(xlabel, dict3['pred_Student'], marker='P')
ax2.plot(xlabel, dict4['pred_Student'], marker='v')
ax2.plot(xlabel, dict5['pred_Student'], marker='s')
ax2.set_ylabel('Not use HE', fontsize=14)
ax2.set_ylim(70, 100)
ax2.set_xticks(xlabel)



plt.xlabel('Model', fontsize=16)
fig.savefig('Case_student_Model.jpg')
fig.show()